In [3]:
import datetime
import numpy as np
import pandas as pd
from sklearn.externals import joblib

CT_DIR = 'save/'
import pickle
def load_obj(name):
    with open(CT_DIR + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [12]:
import nltk
from nltk.corpus import stopwords
mystops = set(stopwords.words("english"))

from bs4 import BeautifulSoup
import re
# Transformation de la feature 'Body'
def body_to_words(value):
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(value, "lxml").get_text() 
    #
    # 2. Remove special characters
    letters_only = re.sub("[^a-zA-Z0-9#]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    # 
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in mystops]
    # 
    # 5. Lemme
    porter = nltk.PorterStemmer()
    meaningful_words = [porter.stem(w) for w in meaningful_words]
    #
    # 6. Join the words back into one string separated by space, and return the result.
    return( " ".join( meaningful_words ))   

In [13]:
# Méthode pour récupérer les meilleurs tags en fonction du pourcentage de la prédiction
def get_best_tags(clf, X, lb, n_tags=5):
    decfun = []
    if hasattr(clf, 'decision_function'):
        decfun = clf.decision_function(X)
    elif hasattr(clf, 'predict_proba'):
        decfun = clf.predict_proba(X)
    else:
        return None

    best_tags = np.argsort(decfun)[:, :-(n_tags+1): -1]

    return lb.classes_[best_tags]

In [24]:
clf = load_obj('ClassifierBest')
mlb = load_obj('MultiLabelBinarizer')

df = pd.DataFrame(columns=['Body', 'Title'])
df.loc[0] = 0

In [27]:
df['Body'][0] = ''
df['Title'][0] = ''

df['TextCleaned'] = df['Title'].apply(body_to_words) + ' ' + df['Body'].apply(body_to_words)

ret = get_best_tags(clf, df['TextCleaned'], mlb)

In [26]:
df['Body'][0] = 'my question on oracle database'
df['Title'][0] = 'my title'

df['TextCleaned'] = df['Title'].apply(body_to_words) + ' ' + df['Body'].apply(body_to_words)

ret = get_best_tags(clf, df['TextCleaned'], mlb)

array([['oracle', 'database', 'asp.net', 'language-agnostic', 'deployment']], dtype=object)

In [30]:
'-'.join(t for t in ret[0])

'upload-dll-inheritance-dom-class'